In [ ]:
import matplotlib.pyplot as plt

import scipy.sparse as spr
import scipy.sparse.linalg as la



from opt_operators import *
from algorithms import *
from pd_algorithms import *
from new_pd_algorithms import *
from misc_for_output import *
from gr_pd_algorithms import *


gen = 100

m = 100
n = 100
np.random.seed(gen)
A = np.random.uniform(-1, 1, [m,n])

m = 100
n = 100
np.random.seed(gen)
A = np.random.normal(0, 1, [m, n])


m = 500
n = 100
np.random.seed(gen)
A = np.random.normal(0, 10, [m, n])


m = 1000
n = 2000
np.random.seed(gen)
A = spr.rand(m,n,0.1, 'csr')

x0 = np.ones(n)/n
y0 = np.ones(m)/m
z0 = np.hstack((x0, y0))



# define general proximal operator. Note that it is independent of
# the dimension, so we will use it for both primal and dual variables

def prox_g(x, rho):
    return proj_simplex(x)

# define primal-dual gap
def J_gap(x,y):
    return max(A.dot(x)) - min(A.T.dot(y))

def J1_gap(y,x):
    return J_gap(x,y)

#L = np.sqrt(np.max(LA.eigh(A.dot(A.T))[0])) # this is for dense matrix
L = np.sqrt(np.max(la.eigsh(A.dot(A.T))[0])) # this is for sparse matrix

# step size for PDA and NPDA
tau = 1./L
sigma = 1./L


phi = 1.618
sqrt_phi = np.sqrt(phi)
prec = 1e-10
beta = 1
y1 = y0 + np.random.random(y0.shape) * prec
# need to fix division in case of zero in the denominator
m = LA.norm(y1 - y0) / LA.norm(A.T.dot(y1 - y0))
tau1 = np.sqrt(1.5/beta) * m  # for  GRPDA-L
tau2 = 1./L* sqrt_phi   # for GRPDA

# for  PDAL and NPDAL
#tau0 = np.sqrt(min(m,n))/LA.norm(A) # for dense matrix
tau0 = np.sqrt(min(m,n))/la.norm(A, 'fro') # for sparse matrix
c=L*np.sqrt(tau*(2/3)*sigma)

T = 1e-25
N = 80000
ans1 = pd(J_gap, prox_g, prox_g, A, x0, y0, 1*sigma, 1*tau, numb_iter=N)
ans2 = pd_linesearch(J_gap, prox_g, prox_g, A, x0,y0, tau0, 1, numb_iter=N)

ans3 = pd_Golden(J_gap, prox_g, prox_g, A, x0, y0, tau2,  tau2,phi, numb_iter=N,tol=T)
ans4 = pd_Golden_linesearch(J_gap, prox_g, prox_g, A, x0, y0, tau1, 1, numb_iter=N,tol=T)

ans5 = pd_new(J_gap, prox_g, prox_g, A,  x0, y0, 2/3*sigma, 1*tau, numb_iter=N)
ans6 = pd_new_linesearch(J_gap, prox_g, prox_g, A, x0,y0, tau0, 1, numb_iter=N)


import matplotlib as mpl
mpl.rc('lines', linewidth=1.5)
mpl.rcParams.update(
    {'font.size': 13, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})
mpl.rcParams['xtick.major.pad'] = 2
mpl.rcParams['ytick.major.pad'] = 2


answers = [ans1,ans2,ans3,ans4,ans5,ans6]
alg = ["PDA","PDAL","GRPDA","GRPDAL","NPDA","NPDAL"]

time_stamp =100
#残差f(Ax)+g(x) - f(Ax^*)-g(x^*的图
a1,a2 = cut_time(ans1[0],ans1[1], t=time_stamp)
b1,b2 = cut_time(ans2[0],ans2[1], t=time_stamp)
c1,c2 = cut_time(ans3[-1],ans3[0], t=time_stamp)
d1,d2 = cut_time(ans4[-2],ans4[0], t=time_stamp)
e1,e2 = cut_time(ans5[0],ans5[1], t=time_stamp)
f1,f2 = cut_time(ans6[0],ans6[1], t=time_stamp)


v_min =min(ans1[1]+ans2[1]+ans3[0]+ans4[0]+ans5[1]+ans6[1])
#使用了find_smallest_energy(answers)函数来找到所有算法中的最小能量值v_min，用于标准化残差。
plt.plot(a1, a2-v_min, 'b', label=alg[0])
plt.plot(b1, b2-v_min, 'r', label=alg[1])
plt.plot(c1, c2-v_min, 'g', label=alg[2])
plt.plot(d1, d2-v_min, 'c', label=alg[3])
plt.plot(e1, e2-v_min, 'y', label=alg[4])
plt.plot(f1, f2-v_min, 'm', label=alg[5])

plt.yscale('log')
#设置了y轴为对数刻度（plt.yscale('log')）
plt.xlabel(u'CPU time, seconds  ')
plt.ylabel(u'PD gap $\\mathcal{G}(x^k,y^k)$')
#添加了x轴和y轴的标签，以及图例（plt.legend()），以帮助解释图表
plt.legend()
#plt.savefig('figures/lasso-.pdf',bbox_inches='tight')
#plt.title(' l1-regularization')
plt.show()